# Visualizing data in the transit_database SQLite database

We know from the work in the other notebooks in this file that this database contains 3 tables with various station details
This notebook will demonstrate ways to manipulate the tables and also a simple visualization of the station's location in a Folium map.

Much of the work here has been created with chatGPT plus

In [18]:
import pandas as pd
import sqlite3
import geopandas as gpd
from shapely.geometry import Point
import folium


### Query for data

We will merge station table with the other two entrances table. Note that doing this means there will be several rows where the station coordinate will be duplicated.

In [19]:

# Create a connection to the SQLite database
conn = sqlite3.connect('transit_database.db')

# SQL query
query = """
SELECT stations.*, 
       station_entrances."entrance_id", 
       entrances."longitude" as entrance_longitude, 
       entrances."latitude" as entrance_latitude
FROM stations
LEFT JOIN station_entrances
ON stations."station_code" = station_entrances."station_code"
LEFT JOIN entrances
ON station_entrances."entrance_id" = entrances."entrance_id"
WHERE stations.region='Klang Valley'
"""

# Load the data into a pandas DataFrame
klang_valley_df = pd.read_sql_query(query, conn)

# Don't forget to close the connection
conn.close()


### Create Geopandas and geometry objects

This part isn't necessary, but I believe that it is good practice to use geopandas when working with geographic data.
This part will convert a normal pandas dataframe to a geopandas dataframe using the station's coordinates as the main geometry.

In [3]:
# Create a new column in your DataFrame for the geographic data
klang_valley_df['geometry'] = [Point(xy) for xy in zip(klang_valley_df.longitude, klang_valley_df.latitude)]

# Convert the DataFrame to a GeoDataFrame
klang_valley_gdf = gpd.GeoDataFrame(klang_valley_df, geometry='geometry')

# Set the coordinate reference system (CRS) to EPSG:4326 (WGS84)
klang_valley_gdf.crs = "EPSG:4326"

#### Visualizing stations in the form of markers using lat,long coordinates columns (geopandas not necessary)

In [4]:
# Create a new DataFrame where each latitude and longitude pair is unique
unique_stations = klang_valley_gdf.drop_duplicates(subset=['latitude', 'longitude'])

# Create a map centered around the average latitude and longitude of the stations
map = folium.Map(location=[unique_stations['latitude'].mean(), unique_stations['longitude'].mean()], zoom_start=13)

# Add a marker for each station
for _, station in unique_stations.iterrows():
    folium.Marker(location=[station['latitude'], station['longitude']], 
                  popup=f"{station['name']} ({station['station_id']})").add_to(map)

# Display the map
map


#### Visualizing stations in the form of markers using geometry object column (must use geopandas)

In [5]:
# Create a new GeoDataFrame where each geometry is unique
unique_stations_gdf = klang_valley_gdf.drop_duplicates(subset=['geometry'])

# Create a map centered around the average latitude and longitude of the stations
klang_valley_map = folium.Map(location=[unique_stations_gdf['geometry'].y.mean(), unique_stations_gdf['geometry'].x.mean()], zoom_start=10)

# Add a marker for each station
for _, row in unique_stations_gdf.iterrows():
    folium.Marker(
        location=[row['geometry'].y, row['geometry'].x],  # Extract latitude and longitude from Point object
        popup=f"Station ID: {row['station_id']}<br>Name: {row['name']}<br>Provider: {row['service_provider_name']}"
    ).add_to(klang_valley_map)

# Display the map
klang_valley_map

#### Visualizing all available station entrances

In [6]:
# Create a new DataFrame where each entrances latitude and longitude pair is available
valid_entrances = klang_valley_gdf.dropna(subset=['entrance_latitude', 'entrance_longitude'])

# Create a map centered around the average latitude and longitude of the stations
map = folium.Map(location=[valid_entrances['entrance_latitude'].mean(), valid_entrances['entrance_longitude'].mean()], zoom_start=13)

# Add a marker for each station
for _, entrance in valid_entrances.iterrows():
    folium.Marker(location=[entrance['entrance_latitude'], entrance['entrance_longitude']], 
                  popup=f"{entrance['name']} ({entrance['station_id']})",
                  icon=folium.Icon(color="red", icon="")
                  ).add_to(map)

# Display the map
map

### Supabase Database

As of August 2023, we have started migration to Supabase. The database has already been created through manually uploading of CSV files.

Here we will try and connect to the Supabase instance and visualize the same data as above.

In [42]:
import os
import pandas as pd
from supabase import create_client, Client

url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)
# Initialize the Supabase client

In [61]:
#Query stations table
response = supabase.table('stations').select("*").eq('region', 'Klang Valley').execute()
data,_ = response
stations_data_supa = pd.DataFrame(data[1])
stations_data_supa.index.name = 'station_id'
stations_data_supa.drop({'station_id'},inplace=True,axis=1)

#Query station entrances table
response = supabase.table('station_entrances').select("*").execute()
data,_ = response
station_entrances_data_supa = pd.DataFrame(data[1])
station_entrances_data_supa.index.name = 'relationship_id'
station_entrances_data_supa.drop({'relationship_id'},inplace=True,axis=1)
station_entrances_data_supa


#Query entrances table
response = supabase.table('entrances').select("*").execute()
data,_ = response
entrances_data_supa = pd.DataFrame(data[1])
entrances_data_supa.index.name = 'entrance_id'
entrances_data_supa.drop({'entrance_id'},inplace=True,axis=1)
entrances_data_supa.rename(columns={
    'longitude': 'entrance_longitude',
    'latitude': 'entrance_latitude'
}, inplace=True)
entrances_data_supa



,entrance_longitude,entrance_latitude,entrance_destination,entrance_name
entrance_id,,,,
0,101.711374,3.145929,None,B
1,101.604952,3.113222,None,None
2,101.644077,3.050650,None,None
3,101.699182,3.138565,None,C
4,101.712717,3.158762,None,None
...,...,...,...,...
211,101.657207,2.949712,Off Persiaran APEC,A
212,101.695226,3.173223,None,D
213,101.731475,3.165190,None,None


In [63]:
# First LEFT JOIN between stations and station_entrances on 'station_code'
merged_df = pd.merge(stations_data_supa, station_entrances_data_supa, on='station_code', how='left')

# Second LEFT JOIN with entrances on 'entrance_id'
final_df = pd.merge(merged_df, entrances_data_supa, on='entrance_id', how='left')
final_df


,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened,entrance_id,station_name,entrance_longitude,entrance_latitude,entrance_destination,entrance_name
0,KL Sentral,KA01,Keretapi Tanah Melayu,3.134603,101.686567,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
1,Kuala Lumpur,KA02,Keretapi Tanah Melayu,3.139513,101.693789,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
2,Bank Negara,KA03,Keretapi Tanah Melayu,3.154542,101.693010,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
3,Putra,KA04,Keretapi Tanah Melayu,3.165005,101.691234,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
4,Mid Valley,KB01,Keretapi Tanah Melayu,3.118528,101.678985,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,Taman Selatan,JS21,Rapid KL,3.026450,101.442469,JS,LRT3,11,Sky Blue,#88cffa,Klang Valley,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
384,Sri Andalas,JS22,Rapid KL,3.015225,101.440441,JS,LRT3,11,Sky Blue,#88cffa,Klang Valley,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
385,Klang Jaya,JS23,Rapid KL,3.005072,101.442081,JS,LRT3,11,Sky Blue,#88cffa,Klang Valley,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
386,Bandar Bukit Tinggi,JS24,Rapid KL,2.993526,101.446175,JS,LRT3,11,Sky Blue,#88cffa,Klang Valley,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
klang_valley_df

,station_id,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened,entrance_id,entrance_longitude,entrance_latitude
0,0,KL Sentral,KA01,Keretapi Tanah Melayu,3.134603,101.686567,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,NaN,NaN,NaN
1,1,Kuala Lumpur,KA02,Keretapi Tanah Melayu,3.139513,101.693789,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,NaN,NaN,NaN
2,2,Bank Negara,KA03,Keretapi Tanah Melayu,3.154542,101.693010,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,NaN,NaN,NaN
3,3,Putra,KA04,Keretapi Tanah Melayu,3.165005,101.691234,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,NaN,NaN,NaN
4,4,Mid Valley,KB01,Keretapi Tanah Melayu,3.118528,101.678985,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,254,Taman Selatan,JS21,Rapid KL,3.026450,101.442469,JS,LRT3,11,Sky Blue,#88cffa,Klang Valley,None,None,None,NaN,NaN,NaN
389,255,Sri Andalas,JS22,Rapid KL,3.015225,101.440441,JS,LRT3,11,Sky Blue,#88cffa,Klang Valley,None,None,None,NaN,NaN,NaN
390,256,Klang Jaya,JS23,Rapid KL,3.005072,101.442081,JS,LRT3,11,Sky Blue,#88cffa,Klang Valley,None,None,None,NaN,NaN,NaN
391,257,Bandar Bukit Tinggi,JS24,Rapid KL,2.993526,101.446175,JS,LRT3,11,Sky Blue,#88cffa,Klang Valley,None,None,None,NaN,NaN,NaN
